In [1]:
!pip install optuna

In [2]:
import random
import os

import numpy as np
import pandas as pd

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import missingno

# device 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print (device)

seed = 42 # seed 값 설정
random.seed(seed) # 파이썬 난수 생성기
os.environ['PYTHONHASHSEED'] = str(seed) # 해시 시크릿값 고정
np.random.seed(seed) # 넘파이 난수 생성기

torch.manual_seed(seed) # 파이토치 CPU 난수 생성기
torch.backends.cudnn.deterministic = True # 확정적 연산 사용 설정
torch.backends.cudnn.benchmark = False   # 벤치마크 기능 사용 해제
torch.backends.cudnn.enabled = False        # cudnn 기능 사용 해제

if device == 'cuda':
    torch.cuda.manual_seed(seed) # 파이토치 GPU 난수 생성기
    torch.cuda.manual_seed_all(seed) # 파이토치 멀티 GPU 난수 생성기

cpu


In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [4]:
import pandas as pd
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors

def calculate_metabolic_stability_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        return [None] * 11  # 제거한 특성의 수만큼 None 값 반환

    logP = Descriptors.MolLogP(mol)
    apka = Descriptors.MolWt(mol)
    num_rotatable_bonds = Descriptors.NumRotatableBonds(mol)
    num_hydrogen_acceptors = Descriptors.NumHAcceptors(mol)
    num_hydrogen_donors = Descriptors.NumHDonors(mol)
    morgan_fingerprint = AllChem.GetHashedMorganFingerprint(mol, 6, nBits=4096)
    morgan_array = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(morgan_fingerprint, morgan_array)
    
    tpsa = Descriptors.TPSA(mol)
    j_index = Descriptors.BalabanJ(mol)
    num_rings = len(Chem.GetSymmSSSR(mol))
    num_aromatic_rings = Descriptors.NumAromaticRings(mol)
    
    molar_refractivity = Descriptors.MolMR(mol)
    
    return logP, apka, num_rotatable_bonds, num_hydrogen_acceptors, num_hydrogen_donors, morgan_array, tpsa, j_index, num_rings, num_aromatic_rings, molar_refractivity

# 기존 코드에 수정된 함수로 새로운 특성 추가
train[[
    'logP', 'apka', 'num_rotatable_bonds', 'num_hydrogen_acceptors',
    'num_hydrogen_donors', 'morgan_fingerprint',
    'tpsa', 'j_index', 'num_rings', 'num_aromatic_rings', 'molar_refractivity'
]] = train['SMILES'].apply(calculate_metabolic_stability_descriptors).apply(pd.Series)

test[[
    'logP', 'apka', 'num_rotatable_bonds', 'num_hydrogen_acceptors',
    'num_hydrogen_donors', 'morgan_fingerprint',
    'tpsa', 'j_index', 'num_rings', 'num_aromatic_rings', 'molar_refractivity'
]] = test['SMILES'].apply(calculate_metabolic_stability_descriptors).apply(pd.Series)

train


,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,apka,num_rotatable_bonds,num_hydrogen_acceptors,num_hydrogen_donors,morgan_fingerprint,tpsa,j_index,num_rings,num_aromatic_rings,molar_refractivity
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,...,400.504,8,6,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",89.13,1.672477,3,3,109.1199
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,...,301.415,2,4,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",45.23,1.963084,3,2,86.8737
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,...,297.366,3,7,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",62.45,1.741211,4,3,85.0270
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,...,494.665,5,7,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",84.22,1.345714,5,3,136.0058
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,...,268.316,1,3,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",42.43,2.022491,3,2,77.1825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,3.409,396.195,3,1,5,3.409,...,396.200,4,5,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",64.74,1.704243,3,2,84.3287
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,1.912,359.381,4,1,3,1.844,...,359.389,3,5,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",85.04,1.976850,4,4,101.6902
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,1.941,261.320,3,1,6,2.124,...,261.325,5,5,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",70.14,2.046226,2,2,74.6974
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.989,284.696,5,1,5,0.989,...,284.699,4,6,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",91.51,1.835328,2,1,67.6249


In [6]:
train['AlogP'].fillna(train['AlogP'].median(), inplace=True)
test['AlogP'].fillna(test['AlogP'].median(), inplace=True)

In [7]:
class CustomDataset(Dataset):
    def __init__(self, data, target_col=None, transform=None, is_test=False):
        self.is_test = is_test
        self.transform = transform
        self.is_test = is_test

        if not self.is_test:
            self.data = data.drop(['id', 'SMILES', 'morgan_fingerprint','MLM', 'HLM'], axis=1)
        else: # test
            self.data = data.drop(['id', 'SMILES', 'morgan_fingerprint'], axis=1)


        if self.transform is not None and not self.is_test:  # 훈련 데이터에만 fit_transform 적용
            self.data = self.transform.fit_transform(self.data)
        elif self.transform is not None and self.is_test:  # 테스트 데이터에는 transform만 적용
            self.data = self.transform.transform(self.data)

        if target_col is not None and not self.is_test:
            self.target = data[target_col]

    def __getitem__(self, index):
        features = self.data[index]

        if hasattr(self, 'target'):
            target = self.target[index]
            return torch.tensor(features).to(device).float(), torch.tensor(target).to(device).float().unsqueeze(dim=-1)
        else:
            return torch.tensor(features).to(device).float()

    def __len__(self):
        return len(self.data)


In [8]:
transform = StandardScaler()
transform.fit(train.drop(['id','SMILES','morgan_fingerprint', 'MLM', 'HLM'], axis=1))

train_MLM = CustomDataset(train, target_col='MLM', transform=transform, is_test=False)
train_HLM = CustomDataset(train, target_col='HLM', transform=transform, is_test=False)

In [9]:
X_MLM_train = train_MLM.data
y_MLM_train = train_MLM.target

X_HLM_train = train_HLM.data
y_HLM_train = train_HLM.target

In [10]:
X_MLM_train

array([[ 0.25978464,  0.46045216,  0.65089366, ..., -0.3353936 ,
         0.40083841,  0.53435756],
       [-0.41833617, -0.78569241, -1.33471042, ..., -0.3353936 ,
        -0.50027519, -0.50468403],
       [-0.77668258, -0.83661321,  0.65089366, ...,  0.52662463,
         0.40083841, -0.59093688],
       ...,
       [-0.56018162, -1.28983214, -0.67284239, ..., -1.19741184,
        -0.50027519, -1.07339609],
       [-1.15244861, -0.99585229,  0.65089366, ..., -1.19741184,
        -1.4013888 , -1.40372764],
       [ 0.92048584, -0.86124986, -1.33471042, ..., -0.3353936 ,
         0.40083841, -0.44801507]])

### MLM (XGBoost)

In [32]:
import optuna
from xgboost import XGBRegressor
import numpy as np
from sklearn.model_selection import cross_val_score

# RMSE를 사용하는 스코어 함수 정의
def custom_scorer(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    return 0.5 * rmse

# Objective 함수 정의
def objective(trial):
    param_space = {
        'nthread': -1,
        'learning_rate': trial.suggest_uniform('learning_rate', 0.0001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1500),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.1, 1.0),
        'gamma': trial.suggest_uniform('gamma', 0, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 6.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0, 6.0),
    }

    # XGBoost 모델 생성
    model = XGBRegressor(**param_space, random_state=42)
    
    # 교차 검증 수행
    scores = cross_val_score(model, X_MLM_train, y_MLM_train, cv=8, scoring=make_scorer(custom_scorer))
    
    # 교차 검증 스코어의 평균을 사용하여 Objective 함수를 평가
    avg_rmse = np.mean(scores)
    print("Average RMSE:", avg_rmse)
    
    return avg_rmse

# Optuna 스터디 생성
study = optuna.create_study(direction='minimize')  # 최소화 문제로 설정

# 최적의 하이퍼파라미터 탐색
study.optimize(objective, n_trials=300)

# 최적의 하이퍼파라미터와 스코어 출력
best_params = study.best_params
best_score = study.best_value
print("Best Params:", best_params)
print("Best Score:", best_score)

# 최적의 하이퍼파라미터를 사용하여 XGBoost 모델 생성
best_MLM_model = XGBRegressor(**best_params)

[I 2023-09-12 21:55:37,530] A new study created in memory with name: no-name-7a3a11b5-3147-4e32-b365-759ec9d42821
/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_904/4157395222.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_904/4157395222.py:19: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_904/4157395222.py:20: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_904/4157395222.py

Average RMSE: 16.496859221585936


[I 2023-09-12 21:56:26,274] Trial 1 finished with value: 16.31501600504219 and parameters: {'learning_rate': 0.036381143159359614, 'n_estimators': 1425, 'max_depth': 4, 'min_child_weight': 14, 'subsample': 0.5076730172318827, 'colsample_bytree': 0.7611934864109888, 'gamma': 0.718509154030909, 'reg_alpha': 1.8273083082442012, 'reg_lambda': 5.397900329218096}. Best is trial 1 with value: 16.31501600504219.


Average RMSE: 16.31501600504219


[I 2023-09-12 21:56:28,402] Trial 2 finished with value: 16.108310071529452 and parameters: {'learning_rate': 0.07560744698343833, 'n_estimators': 483, 'max_depth': 4, 'min_child_weight': 13, 'subsample': 0.6944857151149616, 'colsample_bytree': 0.14455341536954897, 'gamma': 0.7921058665384122, 'reg_alpha': 0.34986864631193026, 'reg_lambda': 5.451607997091552}. Best is trial 2 with value: 16.108310071529452.


...

Average RMSE: 16.08374204557525


[I 2023-09-12 22:20:55,205] Trial 298 finished with value: 15.688705818168263 and parameters: {'learning_rate': 0.007015376195097149, 'n_estimators': 1068, 'max_depth': 3, 'min_child_weight': 4, 'subsample': 0.5212100464012823, 'colsample_bytree': 0.8546506540925894, 'gamma': 0.8086086992647196, 'reg_alpha': 3.141694446209817, 'reg_lambda': 0.10999814496972267}. Best is trial 238 with value: 15.669950038150983.


Average RMSE: 15.688705818168263


[I 2023-09-12 22:21:01,575] Trial 299 finished with value: 15.680416772716912 and parameters: {'learning_rate': 0.010086245041179113, 'n_estimators': 762, 'max_depth': 3, 'min_child_weight': 6, 'subsample': 0.537720029129919, 'colsample_bytree': 0.8782530259927097, 'gamma': 0.8362414958777381, 'reg_alpha': 3.9851924426154355, 'reg_lambda': 0.06837359918307387}. Best is trial 238 with value: 15.669950038150983.


Average RMSE: 15.680416772716912
Best Params: {'learning_rate': 0.011527432539888458, 'n_estimators': 713, 'max_depth': 3, 'min_child_weight': 5, 'subsample': 0.5918263126016217, 'colsample_bytree': 0.9469803290592722, 'gamma': 0.6220557386400593, 'reg_alpha': 3.4284359682923506, 'reg_lambda': 3.6124366932321923}
Best Score: 15.669950038150983


In [33]:
# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
optuna.visualization.plot_param_importances(study)

In [34]:
# 하이퍼파라미터 최적화 과정을 확인
optuna.visualization.plot_optimization_history(study)

In [35]:
# 데이터 분할 및 모델 훈련
X_train, X_valid, y_train, y_valid = train_test_split(X_MLM_train, y_MLM_train, test_size=0.3, random_state=42)

# 교차 검증 수행
scores = cross_val_score(best_MLM_model, X_MLM_train, y_MLM_train, cv=8, scoring=make_scorer(custom_scorer))

# 교차 검증 스코어 출력
print("Cross-Validation RMSE Scores:", scores)
print("Average RMSE:", np.mean(scores))

Cross-Validation RMSE Scores: [15.27073229 15.50511115 15.45138287 15.98566811 15.44286752 16.25383133
 15.96823424 15.63079365]
Average RMSE: 15.68857764472357


### HLM (XGBoost)

In [25]:
import optuna
from xgboost import XGBRegressor
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

# RMSE를 사용하는 스코어 함수 정의
def custom_scorer(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    return 0.5 * rmse

# Objective 함수 정의
def objective(trial):
    param_space = {
        'nthread': -1,
        'learning_rate': trial.suggest_uniform('learning_rate', 0.0001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.1, 0.6),
        'gamma': trial.suggest_uniform('gamma', 0, 0.2),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 2.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0, 3.0),
    }


    # 데이터 분할
    X_train, X_valid, y_train, y_valid = train_test_split(X_HLM_train, y_HLM_train, test_size=0.2, random_state=42)

    
    # XGBoost 모델 생성
    model = XGBRegressor(**param_space, random_state=42)
    
    # 교차 검증 수행
    scores = cross_val_score(model, X_MLM_train, y_MLM_train, cv=10, scoring=make_scorer(custom_scorer))
    
    # 교차 검증 스코어의 평균을 사용하여 Objective 함수를 평가
    avg_rmse = np.mean(scores)
    print("Average RMSE:", avg_rmse)
    
    return avg_rmse

# Optuna 스터디 생성
study = optuna.create_study(direction='minimize')  # 최소화 문제로 설정

# 최적의 하이퍼파라미터 탐색
study.optimize(objective, n_trials=500)

# 최적의 하이퍼파라미터와 스코어 출력
best_params = study.best_params
best_score = study.best_value
print("Best Params:", best_params)
print("Best Score:", best_score)

# 최적의 하이퍼파라미터를 사용하여 XGBoost 모델 생성
best_HLM_model = XGBRegressor(**best_params)

[I 2023-09-12 16:19:05,713] A new study created in memory with name: no-name-cdadde1d-8c93-414a-9e0c-2f908cb6756b
/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_904/3536584456.py:17: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_904/3536584456.py:21: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_904/3536584456.py:22: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_904/3536584456.py

Average RMSE: 16.372239198356997


[I 2023-09-12 16:19:16,903] Trial 1 finished with value: 15.748025415234988 and parameters: {'learning_rate': 0.015727315382350535, 'n_estimators': 360, 'max_depth': 7, 'min_child_weight': 5, 'subsample': 0.6531095469991808, 'colsample_bytree': 0.5912161068516135, 'gamma': 0.09457657465282086, 'reg_alpha': 1.3285899372712902, 'reg_lambda': 2.288709896259048}. Best is trial 1 with value: 15.748025415234988.


Average RMSE: 15.748025415234988


[I 2023-09-12 16:19:38,023] Trial 2 finished with value: 16.06496047379583 and parameters: {'learning_rate': 0.013677745244131997, 'n_estimators': 985, 'max_depth': 10, 'min_child_weight': 7, 'subsample': 0.6342076236207029, 'colsample_bytree': 0.5667906041555929, 'gamma': 0.1999954248220039, 'reg_alpha': 0.6334204686423142, 'reg_lambda': 0.718214115234179}. Best is trial 1 with value: 15.748025415234988.


...

Average RMSE: 15.948856744329907


[I 2023-09-12 21:51:39,349] Trial 435 finished with value: 15.651593740299495 and parameters: {'learning_rate': 0.008626095266321998, 'n_estimators': 992, 'max_depth': 4, 'min_child_weight': 10, 'subsample': 0.5006471893019847, 'colsample_bytree': 0.5572555199118823, 'gamma': 0.013087017154846066, 'reg_alpha': 1.5554660710605817, 'reg_lambda': 2.243343889792811}. Best is trial 162 with value: 15.6239527593516.


Average RMSE: 15.651593740299495


[W 2023-09-12 21:51:40,442] Trial 436 failed with parameters: {'learning_rate': 0.0534001739899876, 'n_estimators': 799, 'max_depth': 4, 'min_child_weight': 10, 'subsample': 0.5298689642095142, 'colsample_bytree': 0.3175001596271089, 'gamma': 0.046987504609393924, 'reg_alpha': 1.9451063548025698, 'reg_lambda': 1.855394970891701} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_904/3536584456.py", line 37, in objective
    scores = cross_val_score(model, X_MLM_train, y_MLM_train, cv=10, scoring=make_scorer(custom_scorer))
  File "/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 515, in cross_val_score
    cv_results = cross_validate(
  File "/opt/homebrew/anaconda3/lib/python3.10/site-pack

KeyboardInterrupt: 

In [ ]:
# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
optuna.visualization.plot_param_importances(study)

In [ ]:
# 하이퍼파라미터 최적화 과정을 확인
optuna.visualization.plot_optimization_history(study)

In [ ]:
# 데이터 분할 및 모델 훈련
X_train, X_valid, y_train, y_valid = train_test_split(X_HLM_train, y_HLM_train, test_size=0.3, random_state=42)

# 교차 검증 수행
scores = cross_val_score(best_HLM_model, X_HLM_train, y_HLM_train, cv=8, scoring=make_scorer(custom_scorer))

# 교차 검증 스코어 출력
print("Cross-Validation RMSE Scores:", scores)
print("Average RMSE:", np.mean(scores))

### MLM (RandomForest)

In [78]:
from sklearn.ensemble import RandomForestRegressor

# RMSE를 사용하는 스코어 함수 정의 (이전과 동일)
def custom_scorer(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    return 0.5 * rmse

# Objective 함수 정의 (랜덤 포레스트용으로 수정 및 추가된 하이퍼파라미터 포함)
def objective(trial):
    param_space = {
        'n_jobs': -1,  # 병렬 처리를 위해 모든 CPU 코어 사용
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_uniform('min_samples_split', 0.1, 1.0),
        'min_samples_leaf': trial.suggest_uniform('min_samples_leaf', 0.1, 0.5),
        'max_features': trial.suggest_uniform('max_features', 0.1, 1.0),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 100),
    }

    model = RandomForestRegressor(**param_space)
    
    # 교차 검증 수행
    scores = cross_val_score(model, X_MLM_train, y_MLM_train, cv=5, scoring=make_scorer(custom_scorer))
    
    # 교차 검증 스코어의 평균을 사용하여 Objective 함수를 평가
    avg_rmse = np.mean(scores)
    print("Average RMSE:", avg_rmse)
    
    return avg_rmse

# Optuna 스터디 생성
study = optuna.create_study(direction='minimize')  # 최소화 문제로 설정

# 최적의 하이퍼파라미터 탐색
study.optimize(objective, n_trials=100)

# 최적의 하이퍼파라미터와 스코어 출력
best_params = study.best_params
best_score = study.best_value
print("Best Params:", best_params)
print("Best Score:", best_score)

# 최적의 하이퍼파라미터를 사용하여 랜덤 포레스트 모델 생성
best_MLM_Rmodel = RandomForestRegressor(**best_params)

[I 2023-09-09 19:25:48,234] A new study created in memory with name: no-name-1da5cb72-a68b-4b29-b057-3e1a981350a1
/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/3390386130.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/3390386130.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/3390386130.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-09 19:25:49,837] Trial 0 finished with value: 17.8426017304273

Average RMSE: 17.842601730427344


/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/3390386130.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/3390386130.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/3390386130.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-09 19:25:50,043] Trial 1 finished with value: 17.842585493168297 and parameters: {'n_estimators': 91, 'max_depth': 9, 'min_samples_split': 0.3269261970521142, 'min_samples_leaf

Average RMSE: 17.842585493168297


/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/3390386130.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/3390386130.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/3390386130.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-09 19:25:50,889] Trial 2 finished with value: 17.64262772854413 and parameters: {'n_estimators': 546, 'max_depth': 10, 'min_samples_split': 0.43927049026481335, 'min_samples_le

...

Average RMSE: 16.338396694033396


/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/3390386130.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/3390386130.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/3390386130.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-09 19:28:22,049] Trial 98 finished with value: 16.331810487672378 and parameters: {'n_estimators': 602, 'max_depth': 9, 'min_samples_split': 0.15584986965389191, 'min_samples_l

Average RMSE: 16.331810487672378


/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/3390386130.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/3390386130.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/3390386130.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-09 19:28:22,698] Trial 99 finished with value: 16.455829729908103 and parameters: {'n_estimators': 246, 'max_depth': 8, 'min_samples_split': 0.10600301069479487, 'min_samples_l

Average RMSE: 16.455829729908103
Best Params: {'n_estimators': 175, 'max_depth': 9, 'min_samples_split': 0.19547005766527825, 'min_samples_leaf': 0.1016771449205634, 'max_features': 0.33676576221591076, 'bootstrap': True, 'max_leaf_nodes': 95}
Best Score: 16.28139630342825


In [79]:
# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
optuna.visualization.plot_param_importances(study)

In [80]:
# 하이퍼파라미터 최적화 과정을 확인
optuna.visualization.plot_optimization_history(study)

In [81]:
from sklearn.metrics import mean_squared_error

# 데이터 분할 및 모델 훈련
X_train, X_valid, y_train, y_valid = train_test_split(X_MLM_train, y_MLM_train, test_size=0.3, random_state=42)

# 모델 훈련
best_MLM_Rmodel.fit(X_train, y_train)

# 예측
y_pred = best_MLM_Rmodel.predict(X_valid)

# 평가 지표 (RMSE) 계산
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print("Validation RMSE:", rmse)

Validation RMSE: 33.36505683028845


### HLM (RandomForest)

In [82]:
from sklearn.ensemble import RandomForestRegressor

# RMSE를 사용하는 스코어 함수 정의 (이전과 동일)
def custom_scorer(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    return 0.5 * rmse

# Objective 함수 정의 (랜덤 포레스트용으로 수정 및 추가된 하이퍼파라미터 포함)
def objective(trial):
    param_space = {
        'n_jobs': -1,  # 병렬 처리를 위해 모든 CPU 코어 사용
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_uniform('min_samples_split', 0.1, 1.0),
        'min_samples_leaf': trial.suggest_uniform('min_samples_leaf', 0.1, 0.5),
        'max_features': trial.suggest_uniform('max_features', 0.1, 1.0),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 100),
    }

    model = RandomForestRegressor(**param_space)
    
    # 교차 검증 수행
    scores = cross_val_score(model, X_HLM_train, y_HLM_train, cv=5, scoring=make_scorer(custom_scorer))
    
    # 교차 검증 스코어의 평균을 사용하여 Objective 함수를 평가
    avg_rmse = np.mean(scores)
    print("Average RMSE:", avg_rmse)
    
    return avg_rmse


# Optuna 스터디 생성
study = optuna.create_study(direction='minimize')  # 최소화 문제로 설정

# 최적의 하이퍼파라미터 탐색
study.optimize(objective, n_trials=100)

# 최적의 하이퍼파라미터와 스코어 출력
best_params = study.best_params
best_score = study.best_value
print("Best Params:", best_params)
print("Best Score:", best_score)

# 최적의 하이퍼파라미터를 사용하여 랜덤 포레스트 모델 생성
best_HLM_Rmodel = RandomForestRegressor(**best_params)

[I 2023-09-09 19:28:23,713] A new study created in memory with name: no-name-36ce52ce-0ee2-4f1c-83e2-94a85db75180
/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/1850882972.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/1850882972.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/1850882972.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-09 19:28:24,783] Trial 0 finished with value: 16.9162238759233

Average RMSE: 16.916223875923368


/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/1850882972.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/1850882972.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/1850882972.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-09 19:28:25,197] Trial 1 finished with value: 16.843164035295416 and parameters: {'n_estimators': 155, 'max_depth': 5, 'min_samples_split': 0.18654314600595084, 'min_samples_le

Average RMSE: 16.843164035295416


/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/1850882972.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/1850882972.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/1850882972.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-09 19:28:25,574] Trial 2 finished with value: 18.03899502255354 and parameters: {'n_estimators': 168, 'max_depth': 8, 'min_samples_split': 0.471783012906034, 'min_samples_leaf'

...

Average RMSE: 16.497476088064694


/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/1850882972.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/1850882972.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/1850882972.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-09 19:30:44,249] Trial 98 finished with value: 16.58109652432242 and parameters: {'n_estimators': 182, 'max_depth': 6, 'min_samples_split': 0.21291061912245424, 'min_samples_le

Average RMSE: 16.58109652432242


/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/1850882972.py:14: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/1850882972.py:15: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

/var/folders/7j/2czlfb0x36j_nmkftnnpyg3r0000gp/T/ipykernel_2841/1850882972.py:16: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-09-09 19:30:44,612] Trial 99 finished with value: 16.623862925112945 and parameters: {'n_estimators': 115, 'max_depth': 5, 'min_samples_split': 0.23953381497890391, 'min_samples_l

Average RMSE: 16.623862925112945
Best Params: {'n_estimators': 263, 'max_depth': 6, 'min_samples_split': 0.15831033009650372, 'min_samples_leaf': 0.10006718268585175, 'max_features': 0.3788951181040163, 'bootstrap': False, 'max_leaf_nodes': 16}
Best Score: 16.464139806262473


In [83]:
# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
optuna.visualization.plot_param_importances(study)

In [84]:
# 하이퍼파라미터 최적화 과정을 확인
optuna.visualization.plot_optimization_history(study)

In [85]:
# 데이터 분할 및 모델 훈련
X_train, X_valid, y_train, y_valid = train_test_split(X_HLM_train, y_HLM_train, test_size=0.3, random_state=42)

# 모델 훈련
best_HLM_Rmodel.fit(X_train, y_train)

# 예측
y_pred = best_HLM_Rmodel.predict(X_valid)

# 평가 지표 (RMSE) 계산
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print("Validation RMSE:", rmse)

Validation RMSE: 33.44406323407569


### 예측값 구하기

In [100]:
test_MLM = CustomDataset(test, target_col=None, transform=transform, is_test=True)
test_HLM = CustomDataset(test, target_col=None, transform=transform, is_test=True)

X_MLM_test = test_MLM.data
X_HLM_test = test_HLM.data

# 최적의 에폭에서 중단된 모델을 사용하여 예측 수행
y_MLM_pred = best_MLM_model.predict(X_MLM_test)
y_HLM_pred = best_HLM_model.predict(X_HLM_test)

#y_MLM_Rpred = best_MLM_Rmodel.predict(X_MLM_test)
#y_HLM_Rpred = best_HLM_Rmodel.predict(X_HLM_test)

#ensemble_MLM_pred = (0.8 * y_MLM_pred + 0.2 * y_MLM_Rpred)
#ensemble_HLM_pred = (0.8 * y_HLM_pred + 0.2 * y_HLM_Rpred)

In [101]:
submission = pd.read_csv('./data/sample_submission.csv')

submission['MLM'] = y_MLM_pred
submission['HLM'] = y_HLM_pred
submission

,id,MLM,HLM
0,TEST_000,28.576383,44.876508
1,TEST_001,55.896920,72.483195
2,TEST_002,37.446006,52.613405
3,TEST_003,47.322636,68.711547
4,TEST_004,62.133796,78.551843
...,...,...,...
478,TEST_478,10.656649,34.115367
479,TEST_479,76.039653,81.008454
480,TEST_480,44.234713,66.215071
481,TEST_481,58.930661,75.394173


In [102]:
submission.to_csv('./submissions/Optuna+XGBoost(NewFeatures)_submission_2.csv', index=False)